[필사 주소](https://www.kaggle.com/code/fahd09/yelp-dataset-surpriseme-recommendation-system/notebook  )

## YELP 데이터셋 필사하기

In [1]:
import os
import re
import string

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df_yelp_business = pd.read_json('data/yelp_academic_dataset_business.json', lines=True)
df_yelp_business.fillna('NA', inplace=True)
# we want to make sure we only work with restaurants -- nothing else
df_yelp_business = df_yelp_business[df_yelp_business['categories'].str.contains('Restaurants')]
print('Final Shape: ',df_yelp_business.shape)

Final Shape:  (57173, 15)


In [3]:
df_yelp_review_iter = pd.read_json('data/yelp_academic_dataset_review.json', chunksize=100000, lines=True)

review 에 있는 데이터의 크기가 너무 크기 때문에 chunk 로 불러와 데이터를 읽고 불필요한 비즈니스에 대한 정보를 사전에 필터링하여 삭제하는 과정을 거친다. 여기에서는 총 다섯개의 chunk로 묶을거고.

In [4]:
df_yelp_review = pd.DataFrame()
i=0
for df in df_yelp_review_iter:
    df = df[df['business_id'].isin(df_yelp_business['business_id'])]
    df_yelp_review = pd.concat([df_yelp_review, df])
    i=i+1
    print(i)
    if i==4: break

1
2
3
4


또한 리뷰 리스트에 있는 비즈니스를 제외하고 나머지를 전부 삭제하는 과정을 거친다.

In [5]:
df_yelp_business = df_yelp_business[df_yelp_business['business_id'].isin(df_yelp_review['business_id'])]

In [6]:
df_yelp_business 

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Apn5Q_b6Nz61Tq4XzPdf9A,Minhas Micro Brewery,,1314 44 Avenue NE,Calgary,AB,T2E 6L6,51.091813,-114.031675,4.0,24,1,"{'BikeParking': 'False', 'BusinessAcceptsCredi...","Tours, Breweries, Pizza, Restaurants, Food, Ho...","{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'..."
1,AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,,,Henderson,NV,89002,35.960734,-114.939821,4.5,3,0,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...","Chicken Wings, Burgers, Caterers, Street Vendo...","{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0..."
2,O8S5hYJ1SMc8fA4QBtVujA,La Bastringue,Rosemont-La Petite-Patrie,1335 rue Beaubien E,Montréal,QC,H2G 1K7,45.540503,-73.5993,4.0,5,0,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...","Breakfast & Brunch, Restaurants, French, Sandw...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
8,8-NRKkPY1UiFXW20WXKiXg,Filiberto's Mexican Food,,1440 N. Dysart Ave,Avondale,AZ,85323,33.448106,-112.341302,2.5,40,1,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...","Mexican, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
10,KapTdGyGs7RK0c68Z6hhhg,Sushi 8,Unionville,4568 Highway 7 E,Markham,ON,L3R 1M5,43.862484,-79.30696,1.5,12,0,"{'GoodForKids': 'True', 'NoiseLevel': 'loud', ...","Restaurants, Japanese","{'Monday': '11:30-23:0', 'Tuesday': '11:30-23:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188560,1UmZSifNBsygSWzbfKiAew,Spaghetti & Company,,5585 Simmons St,North Las Vegas,NV,89031,36.261224,-115.180262,3.5,20,0,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...","Restaurants, Italian","{'Monday': '15:0-21:0', 'Tuesday': '15:0-21:0'..."
188567,0zGeYN8ReYwVj27U7EHOJA,American Cafe,The Strip,3049 Las Vegas Blvd S,Las Vegas,NV,89109,36.131516,-115.164565,2.5,7,1,"{'Ambience': '{'romantic': False, 'intimate': ...","Delis, Restaurants, American (Traditional)",NA
188571,G8mz50lT484Ak6qP2N-whQ,Mary Brown's Famous Chicken & Tater,,3199 Dufferin St,North York,ON,M6A 2T2,43.718455,-79.455328,3.0,20,1,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...","Chicken Wings, Fast Food, Restaurants","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
188577,kbaXNZLUyVuWbeQxH-NNKw,Bell's BBQ,Anthem,"10895 S Eastern Ave, Ste 130",Henderson,NV,89052,35.993777,-115.101582,4.0,447,1,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...","American (Traditional), American (New), Restau...","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."


In [7]:
print('Final businesses shape: ', df_yelp_business.shape)
print('Final review shape: ', df_yelp_review.shape)

Final businesses shape:  (38726, 15)
Final review shape:  (235669, 9)


더 효율적으로 데이터를 이용하기 위해 [사이트링크](https://github.com/msahamed/yelp_comments_classification_nlp/blob/master/word_embeddings.ipynb) 에서 다음의 텍스트 가공 함수를 가져와 이용한다.

In [8]:
def clean_text(text):
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)    
    return text

함수를 불러왔으므로 다음과 과정을 apply를 이용하여 데이터에 적용을 해준다.

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/work/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
%%time
df_yelp_review['text'] = df_yelp_review['text'].apply(clean_text)

CPU times: user 52 s, sys: 2.19 s, total: 54.2 s
Wall time: 54.3 s


우리는 리뷰와 카테고리를 벡터화하도록 하겠다.

In [11]:
vectorizer_reviews = CountVectorizer(min_df = .01,max_df = .99, tokenizer = WordPunctTokenizer().tokenize)
vectorized_reviews = vectorizer_reviews.fit_transform(df_yelp_review['text'])

In [12]:
print(vectorized_reviews.shape)

(235669, 858)


상위 100개의 단어를 살펴보자!

In [13]:
' | '.join(vectorizer_reviews.get_feature_names()[:100]) # only the first 100

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'! | + | - | 00 | 1 | 10 | 12 | 15 | 2 | 20 | 3 | 30 | 4 | 5 | 50 | 6 | 7 | 8 | : | ; | a | able | about | absolutely | across | actually | add | added | again | ago | all | almost | along | already | also | although | always | am | amazing | ambiance | amount | and | another | anyone | anything | anyway | anywhere | appetizer | appetizers | are | area | around | arrived | as | asian | ask | asked | asking | ate | atmosphere | attention | attentive | attitude | authentic | available | average | away | awesome | awful | back | bacon | bad | baked | bar | barely | bartender | based | basically | bbq | be | beans | beautiful | beef | beer | beers | before | behind | believe | best | better | big | bill | birthday | bit | bite | black | bland | bowl | boyfriend | bread'

In [14]:
vectorizer_categories = CountVectorizer(min_df = 1, max_df = 1., tokenizer = lambda x: x.split(', '))
vectorized_categories = vectorizer_categories.fit_transform(df_yelp_business['categories'])

In [15]:
print(vectorized_categories.shape)

(38726, 623)


100개의 카테고리를 살펴보자

In [16]:
' | '.join(vectorizer_categories.get_feature_names()[:100]) # only the first 100

'& probates | acai bowls | accessories | active life | acupuncture | adult education | adult entertainment | advertising | afghan | african | airport lounges | airport shuttles | airports | alternative medicine | amateur sports teams | american (new) | american (traditional) | amusement parks | animal shelters | antiques | appliances | appliances & repair | aquariums | arabian | arcades | argentine | armenian | art classes | art galleries | art museums | art schools | arts & crafts | arts & entertainment | asian fusion | australian | austrian | auto customization | auto detailing | auto glass services | auto insurance | auto parts & supplies | auto repair | auto upholstery | automotive | baby gear & furniture | bagels | baguettes | bakeries | bangladeshi | bankruptcy law | bar crawl | barbeque | barbers | bars | bartenders | basque | bavarian | beaches | beauty & spas | bed & breakfast | beer | beer bar | beer garden | beer gardens | beer hall | belgian | beverage store | bike rentals 

 sparse representations 를 이용하여 dot product의 효율을 높이도록 하겠다.

In [17]:
%%time
from scipy import sparse
businessxreview = sparse.csr_matrix(pd.get_dummies(df_yelp_review['business_id']).values)

CPU times: user 1min 42s, sys: 1.93 s, total: 1min 44s
Wall time: 1min 44s


각 행렬의 크기를 확인해보도록 한다.

In [18]:
print('restuarants x categories: \t', vectorized_categories.shape) 
print('restuarants x reviews: \t\t' , businessxreview.shape) 
print('reviews x words: \t\t', vectorized_reviews.shape)

restuarants x categories: 	 (38726, 623)
restuarants x reviews: 		 (235669, 38726)
reviews x words: 		 (235669, 858)


이제 seed 식당을 선택하고 seed 식당만큼 좋을지도 모르는 다른 식당들을 찾을 준비가 되었다. 반드시 리뷰와 평점이 좋은 식당을 선택해야 한다.

In [19]:
# to choose a restaurant, just copy the business id and paste it in the next cell
# you can always rerun the cell to choose another restuarant. 
df_yelp_business.sample(10)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
38575,NECnXyZ8SCjQICOBhG7wTg,The Original Gyro Grill,The Annex,4 Walmer Road,Toronto,ON,M5S 1X2,43.666581,-79.405812,4.0,106,1,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...","Greek, Fast Food, Restaurants","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
75728,welW9Uwq_GZ1DOU7GEe7vQ,Bojangles,,1301 S Cannon Blvd,Kannapolis,NC,28081,35.477183,-80.610966,3.0,5,1,"{'Ambience': '{'romantic': False, 'intimate': ...","Chicken Wings, Restaurants",NA
17128,qW7lM3Jo0_Ctz3X0KYNtCQ,Fresco,,"2820 S Alma School Rd, 10",Chandler,AZ,85249,33.264433,-111.8601,4.5,18,0,"{'Alcohol': 'none', 'BusinessAcceptsCreditCard...","Food, Korean, Restaurants, Sandwiches, Ice Cre...",NA
64508,WoiOpMEcbAfOqNYXqDwm3Q,La Gourmandine Bakery & Pastry Shop,,300 Cochran Rd,Mt Lebanon,PA,15228,40.380645,-80.052417,4.5,58,1,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...","Restaurants, Bakeries, Breakfast & Brunch, Pat...","{'Monday': '7:30-17:0', 'Tuesday': '7:30-17:0'..."
74372,hNSvyreISNd9XXrHyyaSjQ,Nirvana,Kensington Market,434 College St,Toronto,ON,M4W 1A7,43.656571,-79.40712,3.5,143,1,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...","Breakfast & Brunch, Asian Fusion, Bars, Nightl...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ..."
130781,5s7WR7AUQDAOXMNKMCdd_Q,Freshii,Niagara,700 King Street W,Toronto,ON,M5V 2Y6,43.644373,-79.403019,2.5,8,1,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...","Vegetarian, Sandwiches, Restaurants, Food","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'..."
160909,PwmaDYHxxTVvT_4KdBQc5Q,Grill Bistro (Le),Ville-Marie,183 Rue Saint-Paul E,Montréal,QC,H2Y 1G8,45.507387,-73.552693,1.5,10,1,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...","American (New), Restaurants, Canadian (New)",NA
15670,ykSlMTRgyWzAmhP7ywc9gg,Fin Sake Bar Oakville,,221 Lakeshore Road E,Oakville,ON,L6J 1H7,43.446221,-79.667981,3.0,6,0,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...","Nightlife, Bars, Restaurants, Cocktail Bars, T...","{'Tuesday': '17:30-23:0', 'Wednesday': '17:30-..."
153413,ZBq7AD95c2kPei76KVewQQ,Texicali Taco & Tequila Bar,Third Ward,900 NC Music Factory,Charlotte,NC,28206,35.23905,-80.845023,2.0,46,0,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...","Nightlife, Bars, Tex-Mex, Mexican, Arts & Ente...","{'Monday': '11:30-15:0', 'Tuesday': '11:30-22:..."
56658,sghUmnrVFGxvldZQT-mtyg,Cicis,,"2270 N 75th Ave, Ste 101",Phoenix,AZ,85035,33.472653,-112.221504,2.5,45,1,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...","Sandwiches, Pizza, Italian, American (Traditio...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [20]:
business_choose = 'aUrOyWFKxKeVXiFzwbTXSA' # vegan, vegetarian, cafes

먼저 몇몇 리뷰들을 꺼내서 보도록 한다.

In [21]:
new_reviews = df_yelp_review.loc[df_yelp_review['business_id'] == business_choose, 'text']

In [22]:
print('\n'.join([r[:100] for r in new_reviews.tolist()])) # restaurant reviews

vegan restaurant amazing ! warm cozy inside ! everyone nice friendly amazing tea not tea bags either
come madison fairly frequently work make point come green owl often possible delicious talk back hom
really good food service even omnivores ! 
really great restaurant ate dinner next day lunch ordered vegan jambalaya first night healthy mind b
sad green owl live milwaukee fantabulous ! great food warm atmosphere nice people what is like + + w
like green owl think little room improvement extensive diverse menu really like atmosphere nice love
fabulous ! incredibly delicious ! great vegan menu abundant number choices tlt great order again fun
never thought i would find good vegan vegetarian joint sis - in - law took here recently turned vega
year ago information might date remember good food was ! + + their kale chips crab cakes chipotle sa
went vegetarians every thing sounded unappetizing fried seitan sounds unappealing want pay dollars f
favorite thing get crabby cake poboy consistentl

카테고리도 꺼내서 살펴본다

In [23]:
new_categories = df_yelp_business.loc[df_yelp_business['business_id'] == business_choose, 'categories']

In [24]:
new_categories.tolist() #  restaurant categories

['Vegetarian, Cafes, Vegan, Restaurants']

여기서, 두 세트의 거리를 계산한다. 모든 리뷰에 대한 평균 벡터화된 리뷰의 상관 거리를 계산하고, 이 카테고리와 다른 모든 카테고리 사이의 상관 거리를 계산한다. 결과를 보면 범주 트릭이 더 명확해진다.

In [25]:
from scipy.spatial.distance import cdist
# find most similar reviews
dists1 = cdist(vectorizer_reviews.transform(new_reviews).todense().mean(axis=0), 
              vectorized_reviews.T.dot(businessxreview).T.todense(), 
               metric='correlation')
# find most similar categories
dists2 = cdist(vectorizer_categories.transform(new_categories).todense().mean(axis=0), 
              vectorized_categories.todense(), 
               metric='correlation')

이제 두 거리 집합을 결합하고 그 평균을 구합니다.

In [26]:
# combine the two vectors in one matrix
dists_together = np.vstack([dists1.ravel(), dists2.ravel()]).T

In [27]:
dists_together

array([[0.68112292, 0.80215126],
       [0.24976913, 0.84130716],
       [0.64411173, 0.5567631 ],
       ...,
       [0.84762579, 0.71528561],
       [0.78104818, 0.78199686],
       [0.88535272, 0.75484653]])

In [28]:
# this is a key cell: how are we going to prioritize ?
dists = dists_together.mean(axis=1)

In [29]:
dists

array([0.74163709, 0.54553814, 0.60043741, ..., 0.7814557 , 0.78152252,
       0.82009963])

Seed 식당에서 가장 가까운 10개 식당을 골라보자.

In [30]:
# select the closest 10
closest = dists.argsort().ravel()[:10]

Seed 식당을 하나 정해보자.

In [31]:
df_yelp_business.loc[df_yelp_business['business_id']== business_choose, ['business_id', 'categories', 'name', 'stars']]

,business_id,categories,name,stars
89849,aUrOyWFKxKeVXiFzwbTXSA,"Vegetarian, Cafes, Vegan, Restaurants",The Green Owl Cafe,4.0


가장 매칭률이 높은 상위의 결과들을 확인해보자!

In [32]:
df_yelp_business.loc[df_yelp_business['business_id'].isin(df_yelp_business['business_id'].iloc[closest]), ['business_id', 'categories', 'name', 'stars']]

,business_id,categories,name,stars
36563,yGCrsq0AYI8WN7goMVLHJA,"Mexican, Vegan, Vegetarian, Restaurants",Yayo Taco,3.5
47957,AaWlhVc96VvdwNGMiW7HeA,"Vegan, Vegetarian, Restaurants",Ashley's,4.0
58566,hM9g6_VR8sdzfvcWNsnUaQ,"Vegan, Coffee & Tea, Cafes, Food, Restaurants,...",Greenbridge Teahouse and Cafe,4.5
60179,MOgPF7lg25Pn0sgzOwNqmA,"Restaurants, Gluten-Free, Vegan, Vegetarian",Gratitude Cafe,3.0
63515,HM5_OwVGW4YEHgyOf56IAQ,"Vegetarian, Restaurants, Vegan",Green Earth Vegetarian Cuisine,4.0
97560,DB1kg0GJZXNrFYZqsxeavg,"Vegan, Vegetarian, Restaurants",Zizis Vegan Take-Out,4.0
106858,jPqrHW9lXfdtQDmm2wzyrw,"Vegetarian, Restaurants, Vegan, Ethiopian",Abay Ethiopian Cuisine,4.0
111406,Tv8uaK1ymZES4vjSpr1unQ,"Vegetarian, Vegan, Restaurants, Indian",Govinda's,3.5
122588,XGteHUmQZNLDKqnZkVBqZQ,"Restaurants, Vegan, Vegetarian",Veggie Paradise,4.5
134452,ho5ClDsYijmOJy9OSUx8UA,"Cafes, Vegan, Vegetarian, Restaurants",D-Beatstro,4.5


이들 중 많은 수가 같은 범주(채식주의자 및 비건)에서 온 것처럼 보이지만, 이러한 범주(에티오피아, 아프리카, 아시아 퓨전 등)에는 상당한 차이가 있다. 주의해야 할 점은, 이 결과들 모두가 높은 평가를 받고 있는 것처럼 보인다는 것이다.